In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
!cp drive/MyDrive/ssne/train.pkl train.pkl
!cp drive/MyDrive/ssne/test_no_target.pkl test_no_target.pkl

In [38]:
import pandas as pd
import sklearn as sklearn
import torch
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [39]:
device = torch.device("cuda")
device

device(type='cuda')

In [40]:
seed = 2137
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [41]:
train_raw = pd.read_pickle("./train.pkl")
random.shuffle(train_raw)

In [42]:
classes_count = torch.zeros(5)
sizes = []
for x in train_raw:
  classes_count[x[1]] += 1
  sizes.append(np.array(x[0]).shape[0])
print(classes_count)
print(f"Avr size before padding: {sum(sizes)/len(sizes)}")

tensor([1630.,  478.,  154.,  441.,  236.])
Avr size before padding: 436.50493365090165


In [43]:
MAX_SEQUENCE_LEN = 3000

music_data = []
label_data = []
for x in train_raw:
  music_tensor = torch.tensor(x[0], dtype=torch.float)
  music_tensor = music_tensor[:MAX_SEQUENCE_LEN]
  music_data.append(music_tensor)
  label_data.append(x[1])
music_data[0]

tensor([ -1.,  -1.,  -1.,   0.,  12., 124., 124., 119.,  29.,  93.,  78.,  12.,
         12.,  12.,  15., 124.,  13., 149., 124.,  12.,  12.,  12.,  12., 112.,
         12., 172., 172., 125., 157.,  76., 159.,  79.,  45.,  78.,  13.,  45.,
         13.,  92., 108., 108., 114.,  47.,  73.,   0.])

In [44]:
music_data_padded = pad_sequence(music_data, batch_first=True, padding_value=0)
assert len(music_data_padded) == len(label_data)
print(f"Size after paddding: {music_data_padded[0].shape[0]}")

Size after paddding: 2000


In [45]:
TRAIN_TEST_RATIO = 0.8
BATCH_SIZE = 32

train_indices = int(TRAIN_TEST_RATIO*len(music_data_padded))

train_set = torch.utils.data.TensorDataset(music_data_padded[:train_indices], torch.tensor(label_data[:train_indices]))
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE)

test_data, test_targets = music_data_padded[train_indices:], label_data[train_indices:]


In [46]:
class LSTMRegressor(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional, dropout=0.4)
    if bidirectional:
      self.bidirectional = 2
    else:
      self.bidirectional = 1
    self.fc = nn.Linear(hidden_size*len(music_data_padded[0])*self.bidirectional, out_size)

  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size)
    state = torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size)
    return hidden, state

  def forward(self, x, hidden):
    x = torch.transpose(x,0,1)
    all_outputs, hidden = self.lstm(x, hidden)
    all_outputs = torch.transpose(all_outputs,0,1)
    out = torch.flatten(all_outputs, 1)
    x = self.fc(out)
    return x, hidden

In [47]:
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(label_data),y=label_data)
class_weights = torch.tensor(class_weights).float()

In [48]:
def count_accuracy():
  with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(test_data.to(device).unsqueeze(2),(hidden, state))
    p = torch.argmax(preds,1).cpu()
    counter = 0
    for i in range(len(test_targets)):
      if p[i] == test_targets[i]:
        counter += 1
    return counter/len(test_targets)

In [49]:
model = LSTMRegressor(1,20,3,5,False).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss(weight = class_weights.to(device))

In [ ]:
model.train()
last_acc = 0
for epoch in range(31):
  for x, targets in train_loader:
    x = x.to(device).unsqueeze(2)
    targets = targets.to(device)
    hidden, state = model.init_hidden(x.size(0))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(x, (hidden,state))
    preds = preds.squeeze(1)
    optimizer.zero_grad()
    loss = loss_fun(preds, targets)
    loss.backward()
    optimizer.step()
  acc = count_accuracy()
  print(f"Epoch: {epoch}, loss: {loss.item():.3} acc: {acc:.3}")
  if acc - last_acc < -0.1:
    break
  last_acc = acc

Epoch: 0, loss: 1.2 acc: 0.468
Epoch: 1, loss: 1.18 acc: 0.486
Epoch: 2, loss: 1.15 acc: 0.592
Epoch: 3, loss: 1.09 acc: 0.634
Epoch: 4, loss: 0.953 acc: 0.646
Epoch: 5, loss: 0.796 acc: 0.641
Epoch: 6, loss: 0.559 acc: 0.631
Epoch: 7, loss: 0.38 acc: 0.631
Epoch: 8, loss: 0.373 acc: 0.636
Epoch: 9, loss: 0.221 acc: 0.626
Epoch: 10, loss: 0.14 acc: 0.626


In [ ]:
def save_results(tensor):
  predictions = tensor.cpu().detach().numpy()
  pd.DataFrame(predictions).to_csv("result.csv",header=False, index=False)

In [ ]:
test_raw = pd.read_pickle("./test_no_target.pkl")